### ******** 1. Import Libraries ********

In [5]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import seaborn as sns
import pandas as pd
import json

### ******** 2. SETUP ENVIRONMENT ********

In [6]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("PyTorch version:", torch.__version__)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU device")



PyTorch version: 2.6.0
Using MPS device


### ******** 3. CONFIGURATION SETTINGS ********

In [ ]:
# Define paths
DATASET_PATH = "../../dataset-resized" 
RESULTS_DIR = "efficientnet_v2_trashnet_baseline_results"

# DATASET_PATH = "../../garbage-dataset-2"  
# RESULTS_DIR = "efficientV2_baseline_garbage_dataset_results"

# DATASET_PATH = "../../archive/images/images" 
# RESULTS_DIR = "efficientnet_v2_baseline_results_kaggle_dataset"

if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Dataset path {DATASET_PATH} does not exist")

# Create results directory
os.makedirs(RESULTS_DIR, exist_ok=True)

LEARNING_RATE = 0.001
BATCH_SIZE = 32
OPTIMIZER = 'Adam'
EPOCHS = 30
DROPOUT_RATE = 0.5
UNFREEZE_LAYERS = 'none'
NUM_WORKERS = 4 

In [9]:
### ******** 3. DATA PREPARATION ********

In [10]:
def get_transforms():
    """Define image transformations for training and validation"""
    train_transform = transforms.Compose([
        transforms.Resize((240, 240)),
        transforms.CenterCrop((224, 224)),
        transforms.RandomHorizontalFlip(),  
        transforms.RandomRotation(10),      
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

def load_data(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
    """Load and prepare the dataset with train/val/test splits"""
    train_transform, val_transform = get_transforms()
    
    try:
        full_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_transform)
        class_names = full_dataset.classes
        num_classes = len(class_names)
        
        total_size = len(full_dataset)
        train_size = int(0.7 * total_size)
        val_size = int(0.2 * total_size)
        test_size = total_size - train_size - val_size
        
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
            full_dataset, [train_size, val_size, test_size]
        )
        
        val_dataset.dataset.transform = val_transform
        test_dataset.dataset.transform = val_transform
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        val_loader = DataLoader(
            val_dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        test_loader = DataLoader(
            test_dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        print(f"Dataset loaded: {len(train_dataset)} training, {len(val_dataset)} validation, {len(test_dataset)} test images")
        print(f"Classes: {class_names}")
        
        # Print dataset distribution
        class_counts = {class_name: 0 for class_name in class_names}
        for _, class_idx in full_dataset.samples:
            class_counts[class_names[class_idx]] += 1
            
        print("Class distribution:")
        for class_name, count in class_counts.items():
            print(f"  {class_name}: {count} images ({count/total_size:.1%})")
        
        return train_loader, val_loader, test_loader, class_names, num_classes
    
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise

### ******** 5. TRAINING SETUP ********

In [11]:
class EarlyStopping:
    """Early stopping to prevent overfitting"""
    def __init__(self, patience=7, verbose=True, path='best_model.pth', delta=0):
        self.patience = patience
        self.verbose = verbose
        self.path = path
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        
    def __call__(self, val_loss, model):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss



### ******** 6. TRAINING FUNCTION ********

In [12]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=EPOCHS, path='model.pth'):
    """Train the model and track performance metrics"""
    early_stopping = EarlyStopping(patience=7, path=path)
    
    # History tracking
    history = {
        'train_loss': [], 'val_loss': [], 
        'train_acc': [], 'val_acc': [], 
        'f1': []
    }
    
    model = model.to(device)
    start_time = time.time()
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        
        for inputs, labels in tqdm(train_loader, desc=f"Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            batch_size = inputs.size(0)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * batch_size
            running_corrects += torch.sum(preds == labels.data).item()
            total_samples += batch_size
        
        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Validation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                batch_size = inputs.size(0)
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item() * batch_size
                running_corrects += torch.sum(preds == labels.data).item()
                total_samples += batch_size
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        history['val_loss'].append(epoch_loss)
        history['val_acc'].append(epoch_acc)
        history['f1'].append(f1)
        
        print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {f1:.4f}')
        
        # Early stopping check
        early_stopping(epoch_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    # Calculate training time
    time_elapsed = time.time() - start_time
    print(f'Training completed in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    
    # Load best model
    try:
        model.load_state_dict(torch.load(early_stopping.path))
        print(f"Loaded best model from {early_stopping.path}")
    except Exception as e:
        print(f"Warning: Could not load best model - {e}")
    
    return model, history


    print(f'Validation Accuracy: {accuracy:.4f}')
    print('\nClassification Report:')
    print(report_str)
    
    if save_path:
        # Save detailed report
        with open(f"{save_path}_report.txt", 'w') as f:
            f.write(f'Validation Accuracy: {accuracy:.4f}\n\n')
            f.write(report_str)
            
        # Save report as JSON for further analysis
        with open(f"{save_path}_report.json", 'w') as f:
            json.dump(report, f, indent=4)
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix.png")
        plt.close()
        
        # Plot normalized confusion matrix
        plt.figure(figsize=(10, 8))
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Normalized Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix_norm.png")
        plt.close()
        
        print(f'Evaluation results saved to {save_path}')
    
    return accuracy, report

### ******** 7. EVALUATION FUNCTION ********

In [13]:
def evaluate_model(model, data_loader, class_names, save_path=None):
    """Evaluate the model and generate detailed metrics and visualizations"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    report_str = classification_report(all_labels, all_preds, target_names=class_names)
    
    # Print report
    print(f'Validation Accuracy: {accuracy:.4f}')
    print('\nClassification Report:')
    print(report_str)
    
    if save_path:
        # Save detailed report
        with open(f"{save_path}_report.txt", 'w') as f:
            f.write(f'Validation Accuracy: {accuracy:.4f}\n\n')
            f.write(report_str)
            
        # Save report as JSON for further analysis
        with open(f"{save_path}_report.json", 'w') as f:
            json.dump(report, f, indent=4)
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix.png")
        plt.close()
        
        # Plot normalized confusion matrix
        plt.figure(figsize=(10, 8))
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Normalized Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix_norm.png")
        plt.close()
        
        print(f'Evaluation results saved to {save_path}')
    
    return accuracy, report

### ******** 8. VISUALIZATION FUNCTION ********

In [14]:
def plot_history(history, save_path=None):
    """Plot training metrics history"""
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 3, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 3, 3)
    plt.plot(history['f1'], label='F1 Score')
    plt.title('F1 Score')
    plt.xlabel('Epoch')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(f"{save_path}_history.png")
        print(f'Training history plot saved to {save_path}_history.png')
    
    plt.close()


### ******** 9. MAIN EXECUTION FUNCTION ********

In [ ]:
def main():
    print("\n" + "="*50)
    print("EfficientNetV2-S Baseline Model for Recycling Material Classification")
    print("="*50)
    
    # Create directory for baseline results
    baseline_dir = os.path.join(RESULTS_DIR, "baseline")
    os.makedirs(baseline_dir, exist_ok=True)
    
    # Load data
    train_loader, val_loader, test_loader, class_names, num_classes = load_data(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
    
    # Create model
    model = create_efficientnet_v2(num_classes, dropout_rate=DROPOUT_RATE)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Model created: EfficientNetV2-S with {num_classes} output classes")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Training strategy: {UNFREEZE_LAYERS if UNFREEZE_LAYERS != 'none' else 'Only classifier trained, backbone frozen'}")
    
    # Setup loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    if OPTIMIZER == 'Adam':
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                              lr=LEARNING_RATE)
    
    # Save configuration 
    config = {
        'model': 'EfficientNetV2-S',
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'optimizer': OPTIMIZER,
        'epochs': EPOCHS,
        'dropout_rate': DROPOUT_RATE,
        'unfreeze_layers': UNFREEZE_LAYERS,
        'num_workers': NUM_WORKERS,
        'total_parameters': total_params,
        'trainable_parameters': trainable_params,
        'device': str(device),
        'dataset_path': DATASET_PATH,
        'num_classes': num_classes,
        'class_names': class_names
    }
    
    with open(os.path.join(baseline_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Train the model
    print("\nStarting model training...")
    model_path = os.path.join(baseline_dir, 'best_model.pth')
    model, history = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=EPOCHS, path=model_path
    )
    
    # Plot and save training history
    history_path = os.path.join(baseline_dir, 'training_history')
    plot_history(history, save_path=history_path)
    
    # Evaluate on validation set
    print("\nEvaluating on validation set:")
    val_results_path = os.path.join(baseline_dir, 'validation_results')
    val_accuracy, val_report = evaluate_model(model, val_loader, class_names, save_path=val_results_path)
    
    # Evaluate on test set
    print("\nEvaluating on test set:")
    test_results_path = os.path.join(baseline_dir, 'test_results')
    test_accuracy, test_report = evaluate_model(model, test_loader, class_names, save_path=test_results_path)
    
    # Save model summary
    model_summary = {
        'model_type': 'EfficientNetV2-S',
        'num_classes': num_classes,
        'class_names': class_names,
        'parameters': total_params,
        'trainable_parameters': trainable_params,
        'val_accuracy': val_accuracy,
        'test_accuracy': test_accuracy,
        'val_f1_score': val_report['weighted avg']['f1-score'],
        'test_f1_score': test_report['weighted avg']['f1-score'],
        'per_class_f1': {cls: test_report[cls]['f1-score'] for cls in class_names}
    }
    
    with open(os.path.join(baseline_dir, 'model_summary.json'), 'w') as f:
        json.dump(model_summary, f, indent=4)
    
    print("\n" + "="*50)
    print("BASELINE MODEL SUMMARY")
    print("="*50)
    print(f"Model: EfficientNetV2-S")
    print(f"Classes: {class_names}")
    print(f"Total Parameters: {model_summary['parameters']:,}")
    print(f"Trainable Parameters: {model_summary['trainable_parameters']:,}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Validation F1 Score: {val_report['weighted avg']['f1-score']:.4f}")
    print(f"Test F1 Score: {test_report['weighted avg']['f1-score']:.4f}")
    print("="*50)
    print(f"Full results saved to {baseline_dir}")
    
    return model, test_accuracy, test_report

if __name__ == "__main__":
    main()


EfficientNetV2-S Baseline Model for Recycling Material Classification
Dataset loaded: 1768 training, 505 validation, 254 test images
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Class distribution:
  cardboard: 403 images (15.9%)
  glass: 501 images (19.8%)
  metal: 410 images (16.2%)
  paper: 594 images (23.5%)
  plastic: 482 images (19.1%)
  trash: 137 images (5.4%)
Model created: EfficientNetV2-S with 6 output classes
Total parameters: 20,185,174
Trainable parameters: 7,686 (0.04%)
Training strategy: Only classifier trained, backbone frozen

Starting model training...
Epoch 1/30


Training: 100%|█████████████████████████████████| 56/56 [00:33<00:00,  1.68it/s]


Train Loss: 1.3574 Acc: 0.5226


Validation: 100%|███████████████████████████████| 16/16 [00:24<00:00,  1.56s/it]


Val Loss: 1.0315 Acc: 0.7366 F1: 0.7198
Validation loss decreased (inf --> 1.031513). Saving model...
Epoch 2/30


...

Val Loss: 0.4640 Acc: 0.8416 F1: 0.8414
EarlyStopping counter: 2 out of 7
Training completed in 29m 4s
Loaded best model from efficientnet_v2_baseline_results/baseline/best_model.pth
Training history plot saved to efficientnet_v2_baseline_results/baseline/training_history_history.png

Evaluating on validation set:


Evaluating: 100%|███████████████████████████████| 16/16 [00:24<00:00,  1.53s/it]


Validation Accuracy: 0.8376

Classification Report:
              precision    recall  f1-score   support

   cardboard       0.91      0.91      0.91        75
       glass       0.78      0.83      0.80        94
       metal       0.81      0.79      0.80        82
       paper       0.89      0.91      0.90       120
     plastic       0.83      0.77      0.80       102
       trash       0.75      0.75      0.75        32

    accuracy                           0.84       505
   macro avg       0.83      0.83      0.83       505
weighted avg       0.84      0.84      0.84       505

Evaluation results saved to efficientnet_v2_baseline_results/baseline/validation_results

Evaluating on test set:


Evaluating: 100%|█████████████████████████████████| 8/8 [00:22<00:00,  2.85s/it]


Validation Accuracy: 0.8150

Classification Report:
              precision    recall  f1-score   support

   cardboard       0.89      0.87      0.88        38
       glass       0.79      0.80      0.80        51
       metal       0.85      0.85      0.85        47
       paper       0.87      0.84      0.85        55
     plastic       0.74      0.79      0.76        47
       trash       0.67      0.62      0.65        16

    accuracy                           0.81       254
   macro avg       0.80      0.80      0.80       254
weighted avg       0.82      0.81      0.82       254

Evaluation results saved to efficientnet_v2_baseline_results/baseline/test_results

BASELINE MODEL SUMMARY
Model: EfficientNetV2-S
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Total Parameters: 20,185,174
Trainable Parameters: 7,686
Validation Accuracy: 0.8376
Test Accuracy: 0.8150
Validation F1 Score: 0.8373
Test F1 Score: 0.8152
Full results saved to efficientnet_v2_baseline_